In [19]:
from utils.mypytable import MyPyTable 
import utils.plot_utils as plot_utils
import importlib
import utils.myclassifiers
importlib.reload(utils.myclassifiers)
from utils.myclassifiers import MyDecisionTreeClassifier
from utils.myclassifiers import MyKNeighborsClassifier
from utils.myclassifiers import MyDummyClassifier
from utils.myclassifiers import MyNaiveBayesClassifier
from utils.myrandomforestclassifier import MyRandomForestClassifier
import random

import utils.myevaluation
importlib.reload(utils.myevaluation)
import utils.myevaluation as myevaluation
from itertools import chain
import utils.myutils as utils
importlib.reload(utils)
importlib.reload(plot_utils)


<module 'utils.plot_utils' from '/home/Project/V2/DataScienceAlgsProject/utils/plot_utils.py'>

In [20]:
cont_table = MyPyTable()
disc_table = MyPyTable()

cont_table.load_from_file('data_continuous.csv')
disc_table.load_from_file('data_discretized.csv')

In [21]:
cont_target_vals = cont_table.get_column('morph')
cont_table.drop_cols(['morph'])

disc_target_vals = disc_table.get_column('morph')
disc_table.drop_cols(['morph'])

In [22]:
# Discretized
X_train_idx_folds, X_test_idx_folds = myevaluation.stratified_kfold_cross_validation(
    disc_table.data, disc_target_vals, n_splits=7)

X_train = [[disc_table.data[idx] for idx in fold] for fold in X_train_idx_folds]
y_train = [[disc_target_vals[idx] for idx in fold]
            for fold in X_train_idx_folds]
X_test = [[disc_table.data[idx] for idx in fold] for fold in X_test_idx_folds]
y_test = [[disc_target_vals[idx] for idx in fold]
            for fold in X_test_idx_folds]

knn = MyKNeighborsClassifier(3)
dummy = MyDummyClassifier()
# bayes = MyNaiveBayesClassifier()
dec_tree = MyDecisionTreeClassifier()
forest = MyRandomForestClassifier()

classifier_names = ['dummy', 'dec_tree', 'forest']
models = [dummy, dec_tree, forest]
preds = [[] for _ in models]

forest.F = 3
forest.M = 50
forest.N = 21

for i in range(len(X_train)):
    for j in range(len(models)):
        models[j].fit(X_train[i], y_train[i])
        preds[j] += models[j].predict(X_test[i])

y_test = list(chain.from_iterable(y_test))

for i in range(len(preds)):
    myevaluation.cross_val_stats(classifier_names[i], y_test, preds[i])

----------
Classifier: dummy
----------
----------
Accuracy: 0.46106011252591056
----------
Error Rate: 0.5389398874740894
----------
Confusion Matrix: [[0, 459, 0, 0], [0, 1557, 0, 0], [0, 444, 0, 0], [0, 917, 0, 0]]
----------
Classifier: dec_tree
----------
----------
Accuracy: 0.4432928634883032
----------
Error Rate: 0.5567071365116968
----------
Confusion Matrix: [[11, 329, 16, 103], [32, 1181, 52, 292], [17, 326, 10, 91], [24, 575, 23, 295]]
----------
Classifier: forest
----------
----------
Accuracy: 0.46194847497779096
----------
Error Rate: 0.5380515250222091
----------
Confusion Matrix: [[0, 447, 0, 12], [0, 1524, 0, 33], [0, 440, 0, 4], [2, 879, 0, 36]]


In [23]:
# Continuous 
X_train_idx_folds, X_test_idx_folds = myevaluation.stratified_kfold_cross_validation(
    cont_table.data, cont_target_vals, n_splits=7)

X_train = [[cont_table.data[idx] for idx in fold] for fold in X_train_idx_folds]
y_train = [[cont_target_vals[idx] for idx in fold]
            for fold in X_train_idx_folds]
X_test = [[cont_table.data[idx] for idx in fold] for fold in X_test_idx_folds]
y_test = [[cont_target_vals[idx] for idx in fold]
            for fold in X_test_idx_folds]

# knn = MyKNeighborsClassifier(3)
dummy = MyDummyClassifier()
bayes = MyNaiveBayesClassifier()
# dec_tree = MyDecisionTreeClassifier()

classifier_names = ['dummy', 'bayes']
models = [dummy, bayes]
preds = [[] for _ in models]

for i in range(len(X_train)):
    # print('i', i)
    for j in range(len(models)):
        # print('j', j)
        models[j].fit(X_train[i], y_train[i])
        preds[j] += models[j].predict(X_test[i])

y_test = list(chain.from_iterable(y_test))

for i in range(len(preds)):
    if classifier_names[i] == 'bayes':
        preds[i] = bayes.convert_idxs_to_class_labels(y_test, preds[i])
        myevaluation.cross_val_stats(classifier_names[i], y_test, preds[i])
    else:
        myevaluation.cross_val_stats(classifier_names[i], y_test, preds[i])

----------
Classifier: dummy
----------
----------
Accuracy: 0.47662575084878556
----------
Error Rate: 0.5233742491512144
----------
Confusion Matrix: [[0, 510, 0, 0], [0, 1825, 0, 0], [0, 496, 0, 0], [0, 998, 0, 0]]
----------
Classifier: bayes
----------
----------
Accuracy: 0.3499608252807522
----------
Error Rate: 0.6500391747192478
----------
Confusion Matrix: [[47, 187, 79, 197], [160, 748, 302, 615], [41, 178, 56, 221], [62, 312, 135, 489]]


In [24]:
# Continuous kNN down selection 
utils.randomize_in_place(cont_table.data, cont_target_vals)
cont_table_copy = cont_table.data.copy()
cont_table_copy = cont_table_copy[:2001]
X_train_idx_folds, X_test_idx_folds = myevaluation.stratified_kfold_cross_validation(
    cont_table_copy, cont_target_vals, n_splits=7)

X_train = [[cont_table_copy[idx] for idx in fold] for fold in X_train_idx_folds]
y_train = [[cont_target_vals[idx] for idx in fold]
            for fold in X_train_idx_folds]
X_test = [[cont_table_copy[idx] for idx in fold] for fold in X_test_idx_folds]
y_test = [[cont_target_vals[idx] for idx in fold]
            for fold in X_test_idx_folds]

knn = MyKNeighborsClassifier(5)
# dummy = MyDummyClassifier()
# bayes = MyNaiveBayesClassifier()
# dec_tree = MyDecisionTreeClassifier()

classifier_names = ['dummy', 'knn']
models = [dummy, knn]
preds = [[] for _ in models]

for i in range(len(X_train)):
    # print('i', i)
    for j in range(len(models)):
        # print('j', j)
        models[j].fit(X_train[i], y_train[i])
        preds[j] += models[j].predict(X_test[i])

y_test = list(chain.from_iterable(y_test))

for i in range(len(preds)):
    myevaluation.cross_val_stats(classifier_names[i], y_test, preds[i])

----------
Classifier: dummy
----------
----------
Accuracy: 0.46276861569215394
----------
Error Rate: 0.537231384307846
----------
Confusion Matrix: [[0, 274, 0, 0], [0, 926, 0, 0], [0, 267, 0, 0], [0, 534, 0, 0]]
----------
Classifier: knn
----------
----------
Accuracy: 0.32083958020989506
----------
Error Rate: 0.6791604197901049
----------
Confusion Matrix: [[7, 136, 58, 73], [196, 552, 0, 178], [24, 189, 5, 49], [82, 334, 40, 78]]
